In [2]:
import json
import os
import shutil
from pathlib import Path
from collections import Counter

# ==============================================================================
# CONFIGURAÇÕES
# ==============================================================================
INPUT_DIR = Path('data/json/cleaned_factors')
OUTLIER_DIR = Path('data/json/Identified_outliers')
LIMITE_OUTLIER = 2  # Se aparecer x vezes ou menos, é considerado outlier

def analisar_e_segregar():
    # 1. Preparação do ambiente
    if not INPUT_DIR.exists():
        print(f"Erro: Diretório de entrada '{INPUT_DIR}' não encontrado.")
        return

    # Cria diretório de destino se não existir
    OUTLIER_DIR.mkdir(parents=True, exist_ok=True)

    arquivos_json = list(INPUT_DIR.glob("*.json"))
    print(f"📂 Lendo {len(arquivos_json)} arquivos para análise de frequência...")

    # ==========================================================================
    # PASSO 1: CONTAGEM GLOBAL (Mapear o território)
    # ==========================================================================
    contador_global = Counter()

    # Dicionário para guardar os fatores de TODOS arquivos em memória
    cache_arquivos = {}

    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)

            # Extrai a lista, garantindo que seja uma lista mesmo se for None
            fatores = dados.get('conteudo', {}).get('fatores_contribuintes', [])
            if fatores is None:
                fatores = []

            # Normalização básica
            fatores = [f.strip() for f in fatores if f]

            # Guardamos no cache INDEPENDENTE de ter fatores ou não
            cache_arquivos[arquivo] = fatores

            # Só contamos para a estatística se houver fatores
            if fatores:
                contador_global.update(fatores)

        except Exception as e:
            print(f"⚠️ Erro ao ler {arquivo.name}: {e}")

    print(f"📊 Total de fatores únicos identificados: {len(contador_global)}")

    # ==========================================================================
    # PASSO 2: IDENTIFICAR OUTLIERS
    # ==========================================================================
    # Labels que aparecem LIMITE_OUTLIER vezes ou menos
    labels_outliers = {
        label for label, freq in contador_global.items()
        if freq <= LIMITE_OUTLIER
    }

    print(f"🔍 Identificados {len(labels_outliers)} labels considerados 'raros' (frequência <= {LIMITE_OUTLIER}).")

    # ==========================================================================
    # PASSO 3: SEGREGAR ARQUIVOS
    # ==========================================================================
    movidos_vazios = 0
    movidos_raros = 0
    total_movidos = 0

    print("\n📦 Iniciando segregação de arquivos...")

    for arquivo_path, fatores in cache_arquivos.items():
        eh_vazio = len(fatores) == 0
        contem_outlier = any(fator in labels_outliers for fator in fatores)

        motivo = None
        if eh_vazio:
            motivo = "Sem Fatores"
            movidos_vazios += 1
        elif contem_outlier:
            motivo = "Fator Raro"
            movidos_raros += 1

        # Se tiver motivo para mover, movemos
        if motivo:
            destino = OUTLIER_DIR / arquivo_path.name
            try:
                shutil.move(str(arquivo_path), str(destino))
                total_movidos += 1
                # Opcional: printar movimentação individual se quiser debugar
                # print(f" -> Movendo {arquivo_path.name} ({motivo})")
            except Exception as e:
                print(f"❌ Erro ao mover {arquivo_path.name}: {e}")

    # ==========================================================================
    # PASSO 4: RELATÓRIO FINAL
    # ==========================================================================
    print("\n" + "="*60)
    print(f"RELATÓRIO DE FATORES COMUNS (Frequência > {LIMITE_OUTLIER})")
    print("="*60)

    fatores_comuns = {
        label: freq for label, freq in contador_global.items()
        if freq > LIMITE_OUTLIER
    }

    for fator, freq in sorted(fatores_comuns.items()):
        print(f"• {fator}: {freq}")

    print("\n" + "="*60)
    print("RESUMO DA OPERAÇÃO")
    print("="*60)
    print(f"📂 Arquivos analisados: {len(arquivos_json)}")
    print(f"📉 Arquivos segregados (Total): {total_movidos}")
    print(f"   ├─ Por ausência de fatores: {movidos_vazios}")
    print(f"   └─ Por conter fatores raros: {movidos_raros}")
    print(f"✅ Arquivos mantidos (Dataset Limpo): {len(arquivos_json) - total_movidos}")
    print(f"📁 Local dos segregados: {OUTLIER_DIR}")
    print("="*60)

if __name__ == "__main__":
    analisar_e_segregar()

📂 Lendo 1567 arquivos para análise de frequência...
📊 Total de fatores únicos identificados: 264
🔍 Identificados 167 labels considerados 'raros' (frequência <= 2).

📦 Iniciando segregação de arquivos...

RELATÓRIO DE FATORES COMUNS (Frequência > 2)
• aplicação do comando: 149
• aplicação do comando (contribuiu): 180
• aplicação do comando (contribuiu) atitude (contribuiu): 3
• aplicação do comando (indeterminado): 89
• atenção: 8
• atenção (contribuiu): 58
• atenção (indeterminado): 39
• atitude: 20
• atitude (contribuiu): 149
• atitude (indeterminado): 64
• atuação no comando (contribuiu): 4
• capacitação e treinamento (contribuiu): 12
• capacitação e treinamento (indeterminado): 23
• característica da tarefa (indeterminado): 4
• comunicação: 10
• comunicação (contribuiu): 7
• comunicação (indeterminado): 4
• condição meteorológica adversa: 18
• condição meteorológica adversa (contribuiu): 30
• condição meteorológica adversa (indeterminado): 35
• coordenação de cabine: 24
• coordenaçã